In [1]:
import pandas as pd
import json
import re

In [2]:
ori_df = pd.read_json('./data_panre.json', lines=True)
ori_df

,id,url,title,doc
0,0,https://glaw.scourt.go.kr/wsjo/panre/sjo100.do...,대법원 2024. 4. 4. 선고 2023다298670 판결 [보수약정금]〈북한주민...,【판시사항】\n[1] 북한주민이 상속 등으로 취득한 재산 그 자체는 물론이고 그 재...
1,1,https://glaw.scourt.go.kr/wsjo/panre/sjo100.do...,대법원 2024. 4. 4. 선고 2022두56661 판결 [입학전형이의신청거부처분...,【판시사항】\n[1] 국립대학교 법학전문대학원 입시 과정에서 제칠일안식일예수재림교 ...
2,2,https://glaw.scourt.go.kr/wsjo/panre/sjo100.do...,대법원 2024. 3. 29.자 2024터2 결정 [피해자보호명령등에대한재항고]〈피...,【판시사항】\n[1] 가정폭력범죄의 처벌 등에 관한 특례법에 도입된 피해자보호명령 ...
3,3,https://glaw.scourt.go.kr/wsjo/panre/sjo100.do...,대법원 2024. 3. 28. 선고 2023다308911 판결 [부당이득금]〈채무를...,【판시사항】\n[1] 채무를 면하는 경우와 같이 어떠한 사실의 발생으로 당연히 발생...
4,4,https://glaw.scourt.go.kr/wsjo/panre/sjo100.do...,대법원 2024. 3. 28. 선고 2023도15133 판결 [성폭력범죄의처벌등에관...,【판시사항】\n[1] 형사소송법 제313조 제1항에서 규정한 ‘전 2조의 규정 이외...
...,...,...,...,...
955,955,https://glaw.scourt.go.kr/wsjo/panre/sjo100.do...,대법원 2023. 4. 27. 선고 2020도6874 판결 [폭행]〈정당방위의 침해...,【판시사항】\n정당방위의 요건 중 ‘침해의 현재성’의 의미 및 일련의 연속되는 행위...
956,956,https://glaw.scourt.go.kr/wsjo/panre/sjo100.do...,대법원 2023. 4. 27. 선고 2022다307003 판결 [임금]〈택시운수종사...,【판시사항】\n택시운송사업자의 운송비용 전가를 금지하는 구 택시운송사업의 발전에 관...
957,957,https://glaw.scourt.go.kr/wsjo/panre/sjo100.do...,대법원 2023. 4. 27. 선고 2019다247903 판결 [건물등철거]〈주한외...,【판시사항】\n[1] 우리나라의 영토 내에서 행하여진 외국의 사법적(사법적) 행위에...
958,958,https://glaw.scourt.go.kr/wsjo/panre/sjo100.do...,대법원 2023. 4. 13. 선고 2021다305338 판결 [근저당권말소]〈물상...,【판시사항】\n[1] 채권자와 채무자 모두가 기한의 이익을 갖는 이자부 금전소비대차...


#### 필요없는 데이터 column drop

In [3]:
dropped_df = ori_df.drop(columns='url')
dropped_df = dropped_df.drop(columns='id')
dropped_df.count()

title    960
doc      960
dtype: int64

#### 판례 title만 분리

In [4]:
df_title1 = dropped_df['title'].str.split(r"\d+ 판결 \[", expand=True)

df_title1.columns = ['날짜', 'title']

df_title1['title'] = '[' + df_title1['title']

In [5]:
df_title1.count()

날짜       960
title    857
dtype: int64

In [6]:
df_title2 = dropped_df['title'].str.split(r"\d+ 결정 \[", expand=True)

df_title2.columns = ['날짜', 'title']

df_title2['title'] = '[' + df_title2['title']

In [7]:
print(df_title2.count())
df_title2

날짜       960
title     57
dtype: int64


,날짜,title
0,대법원 2024. 4. 4. 선고 2023다298670 판결 [보수약정금]〈북한주민...,NaN
1,대법원 2024. 4. 4. 선고 2022두56661 판결 [입학전형이의신청거부처분...,NaN
2,대법원 2024. 3. 29.자 2024터,[피해자보호명령등에대한재항고]〈피해자보호명령의 발령 요건 및 심리절차의 법령 위반이...
3,대법원 2024. 3. 28. 선고 2023다308911 판결 [부당이득금]〈채무를...,NaN
4,대법원 2024. 3. 28. 선고 2023도15133 판결 [성폭력범죄의처벌등에관...,NaN
...,...,...
955,대법원 2023. 4. 27. 선고 2020도6874 판결 [폭행]〈정당방위의 침해...,NaN
956,대법원 2023. 4. 27. 선고 2022다307003 판결 [임금]〈택시운수종사...,NaN
957,대법원 2023. 4. 27. 선고 2019다247903 판결 [건물등철거]〈주한외...,NaN
958,대법원 2023. 4. 13. 선고 2021다305338 판결 [근저당권말소]〈물상...,NaN


In [8]:
df_title1['title'] = df_title1['title'].fillna(df_title2['title'])

In [9]:
df_title1.count()

날짜       960
title    914
dtype: int64

#### 판결요지 or 결정요지 분리

In [10]:
df_split1 = dropped_df['doc'].str.split('【판결요지】', expand=True)

df_split1.columns = ['판시사항', '판결요지']

In [11]:
df_split2 = dropped_df['doc'].str.split('【결정요지】', expand=True)

df_split2.columns = ['판시사항', '결정요지']

In [12]:
df_split2.count('rows')

판시사항    960
결정요지     62
dtype: int64

In [13]:
df_split1['판결요지'] = df_split1['판결요지'].fillna(df_split2['결정요지'])
print(df_split1.count())
df_split1

판시사항    960
판결요지    952
dtype: int64


,판시사항,판결요지
0,【판시사항】\n[1] 북한주민이 상속 등으로 취득한 재산 그 자체는 물론이고 그 재...,\n[1] 남북 주민 사이의 가족관계와 상속 등에 관한 특례법(이하 ‘남북가족특례법...
1,【판시사항】\n[1] 국립대학교 법학전문대학원 입시 과정에서 제칠일안식일예수재림교 ...,\n[1] 국립대학교 총장은 공권력을 행사하는 주체이자 기본권 수범자로서의 지위를 ...
2,【판시사항】\n[1] 가정폭력범죄의 처벌 등에 관한 특례법에 도입된 피해자보호명령 ...,\n[1] 가정폭력범죄의 처벌 등에 관한 특례법(이하 ‘가정폭력처벌법’이라고 한다)...
3,【판시사항】\n[1] 채무를 면하는 경우와 같이 어떠한 사실의 발생으로 당연히 발생...,\n[1] 법률상 원인 없이 타인의 재산 또는 노무로 인하여 이익을 얻고 이로 인하...
4,【판시사항】\n[1] 형사소송법 제313조 제1항에서 규정한 ‘전 2조의 규정 이외...,\n[1] 헌법 제12조 제1항이 규정한 적법절차의 원칙과 헌법 제27조에 의하여 ...
...,...,...
955,【판시사항】\n정당방위의 요건 중 ‘침해의 현재성’의 의미 및 일련의 연속되는 행위...,\n형법 제21조 제1항은 “현재의 부당한 침해로부터 자기 또는 타인의 법익을 방위...
956,【판시사항】\n택시운송사업자의 운송비용 전가를 금지하는 구 택시운송사업의 발전에 관...,\n구 택시운송사업의 발전에 관한 법률(2020. 6. 9. 법률 제17453호로 ...
957,【판시사항】\n[1] 우리나라의 영토 내에서 행하여진 외국의 사법적(사법적) 행위에...,\n[1] 국제관습법에 의하면 국가의 주권적 행위는 다른 국가의 재판권으로부터 면제...
958,【판시사항】\n[1] 채권자와 채무자 모두가 기한의 이익을 갖는 이자부 금전소비대차...,"\n[1] 기한의 이익은 포기할 수 있으나, 상대방의 이익을 해하지 못한다(민법 제..."


#### 판결 이유 & 결론 분리

In [14]:
df_split3 = df_split1['판결요지'].str.split('【이 유】', expand=True)

df_split3.columns = ['판결요지', '이유']

In [15]:
df_split4 = df_split3['이유'].str.split(r"\d+\.\s*결론", expand=True)

df_split4.columns = ['이유', '결론']

df_split6 = df_split4['결론'].str.split("\n대법관 ", expand=True)

df_split6.columns = ['결론', '그외']

In [16]:
df_split4

,이유,결론
0,\n상고이유(상고이유서 제출기간이 지난 다음 제출된 상고이유보충서의 기재는 상고이유...,"\n원심판결을 파기하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로 하여, ..."
1,\n상고이유를 판단한다.\n1. 사안의 개요\n원심판결 이유와 기록에 따르면 다음 ...,"\n원심판결 중 입학전형이의신청거부처분 취소청구 부분을 파기하되, 이 부분은 이 법..."
2,\n재항고이유를 판단한다.\n1. 피해자보호명령의 발령 요건을 갖추지 못하였다는 주...,"\n그러므로 원심결정을 취소하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로..."
3,\n상고이유를 판단한다.\n1. 법률상 원인 없이 타인의 재산 또는 노무로 인하여 ...,None
4,\n상고이유를 판단한다.\n1. 검사의 상고이유에 관하여\n가. ‘피해자 진술분석 ...,"\n그러므로 상고를 모두 기각하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같..."
...,...,...
955,\n상고이유를 판단한다.\n1. 피고인 1의 상고이유에 대한 판단\n원심은 판시와 ...,"\n그러므로 원심판결 중 피고인 2에 대한 부분을 파기하고, 이 부분 사건을 다시 ..."
956,\n상고이유를 판단한다.\n1. 강행규정 관련 상고이유에 관한 판단\n가. 구 「택...,"\n그러므로 상고를 기각하고 상고비용은 패소자가 부담하도록 하여, 대법관의 일치된 ..."
957,\n상고이유를 판단한다.\n1. 사안의 개요\n원심판결 이유와 기록에 따르면 다음과...,\n주위적 청구에 관한 원고의 상고를 일부 받아들이는 이상 예비적 청구에 관한 피고...
958,"\n상고이유를 판단한다.\n1. 관련 법리\n가. 기한의 이익은 포기할 수 있으나,...",\n그러므로 나머지 상고이유에 대한 판단을 생략한 채 원심판결을 파기하고 사건을 다...


#### 판시사항 분리

In [17]:
df_split5 = df_split1['판시사항'].str.split('【참조조문】', expand=True)

df_split5.columns = ['판시사항', '그 외']

print(df_split5.count())
df_split5['판시사항'][3]

판시사항    960
그 외      62
dtype: int64


'【판시사항】\n[1] 채무를 면하는 경우와 같이 어떠한 사실의 발생으로 당연히 발생하였을 손실을 보지 않는 것이 부당이득이 성립하기 위한 요건인 ‘이익’에 해당하는지 여부(적극)\n[2] 갑의 자녀를 사칭한 성명불상자가 갑에게 전화하여 갑의 휴대전화에 원격조종 프로그램을 설치한 다음, 갑의 은행계좌에서 을에게 부여된 병 주식회사의 가상계좌로 100만 원을 이체하였는데, 위 돈은 을의 병 회사에 대한 신용카드대금으로 결제되었고, 이에 갑이 성명불상자로부터 송금을 받은 을을 상대로 부당이득반환을 구한 사안에서, 을이 위 돈을 사실상 지배할 수 있는 상태에 이르지 못하여 실질적인 이득을 얻지 못하였다는 이유로 을의 부당이득 반환의무를 부정한 원심판결에 소액사건심판법 제3조 제1호에서 정한 ‘대법원의 판례에 상반되는 판단’을 한 잘못이 있다고 한 사례\n'

In [18]:
df_split5['판시사항'] = df_split5['판시사항'].str.replace('【판시사항】\n', '')

print(df_split5.count())
df_split5['판시사항'][3]

판시사항    960
그 외      62
dtype: int64


'[1] 채무를 면하는 경우와 같이 어떠한 사실의 발생으로 당연히 발생하였을 손실을 보지 않는 것이 부당이득이 성립하기 위한 요건인 ‘이익’에 해당하는지 여부(적극)\n[2] 갑의 자녀를 사칭한 성명불상자가 갑에게 전화하여 갑의 휴대전화에 원격조종 프로그램을 설치한 다음, 갑의 은행계좌에서 을에게 부여된 병 주식회사의 가상계좌로 100만 원을 이체하였는데, 위 돈은 을의 병 회사에 대한 신용카드대금으로 결제되었고, 이에 갑이 성명불상자로부터 송금을 받은 을을 상대로 부당이득반환을 구한 사안에서, 을이 위 돈을 사실상 지배할 수 있는 상태에 이르지 못하여 실질적인 이득을 얻지 못하였다는 이유로 을의 부당이득 반환의무를 부정한 원심판결에 소액사건심판법 제3조 제1호에서 정한 ‘대법원의 판례에 상반되는 판단’을 한 잘못이 있다고 한 사례\n'

#### 이유, 결론 확인

In [19]:
df_split4

,이유,결론
0,\n상고이유(상고이유서 제출기간이 지난 다음 제출된 상고이유보충서의 기재는 상고이유...,"\n원심판결을 파기하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로 하여, ..."
1,\n상고이유를 판단한다.\n1. 사안의 개요\n원심판결 이유와 기록에 따르면 다음 ...,"\n원심판결 중 입학전형이의신청거부처분 취소청구 부분을 파기하되, 이 부분은 이 법..."
2,\n재항고이유를 판단한다.\n1. 피해자보호명령의 발령 요건을 갖추지 못하였다는 주...,"\n그러므로 원심결정을 취소하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로..."
3,\n상고이유를 판단한다.\n1. 법률상 원인 없이 타인의 재산 또는 노무로 인하여 ...,None
4,\n상고이유를 판단한다.\n1. 검사의 상고이유에 관하여\n가. ‘피해자 진술분석 ...,"\n그러므로 상고를 모두 기각하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같..."
...,...,...
955,\n상고이유를 판단한다.\n1. 피고인 1의 상고이유에 대한 판단\n원심은 판시와 ...,"\n그러므로 원심판결 중 피고인 2에 대한 부분을 파기하고, 이 부분 사건을 다시 ..."
956,\n상고이유를 판단한다.\n1. 강행규정 관련 상고이유에 관한 판단\n가. 구 「택...,"\n그러므로 상고를 기각하고 상고비용은 패소자가 부담하도록 하여, 대법관의 일치된 ..."
957,\n상고이유를 판단한다.\n1. 사안의 개요\n원심판결 이유와 기록에 따르면 다음과...,\n주위적 청구에 관한 원고의 상고를 일부 받아들이는 이상 예비적 청구에 관한 피고...
958,"\n상고이유를 판단한다.\n1. 관련 법리\n가. 기한의 이익은 포기할 수 있으나,...",\n그러므로 나머지 상고이유에 대한 판단을 생략한 채 원심판결을 파기하고 사건을 다...


In [20]:
df_split4.count('rows')

이유    952
결론    764
dtype: int64

In [21]:
df_split_result = df_split4['결론'].str.split('\n대법관', expand=True)

df_split_result.columns = ['결론', '그외']

print(df_split_result.count())
df_split_result

결론    764
그외    718
dtype: int64


,결론,그외
0,"\n원심판결을 파기하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로 하여, ...",노태악(재판장) 김선수 오경미 서경환(주심)
1,"\n원심판결 중 입학전형이의신청거부처분 취소청구 부분을 파기하되, 이 부분은 이 법...",오경미(재판장) 김선수(주심) 노태악 서경환
2,"\n그러므로 원심결정을 취소하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로...",권영준(재판장) 이동원(주심) 김상환 신숙희
3,None,None
4,"\n그러므로 상고를 모두 기각하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같...",이동원(재판장) 김상환 권영준(주심) 신숙희
...,...,...
955,"\n그러므로 원심판결 중 피고인 2에 대한 부분을 파기하고, 이 부분 사건을 다시 ...",천대엽(재판장) 조재연 민유숙(주심) 이동원
956,"\n그러므로 상고를 기각하고 상고비용은 패소자가 부담하도록 하여, 대법관의 일치된 ...",오경미(재판장) 박정화 김선수(주심) 노태악
957,\n주위적 청구에 관한 원고의 상고를 일부 받아들이는 이상 예비적 청구에 관한 피고...,오석준(재판장) 안철상 노정희(주심) 이흥구
958,\n그러므로 나머지 상고이유에 대한 판단을 생략한 채 원심판결을 파기하고 사건을 다...,천대엽(재판장) 조재연 민유숙(주심) 이동원


In [22]:
df_split_result['결론'] = df_split_result['결론'].str.cat(df_split4['이유'])
df_split_result['결론'] = df_split_result['결론'].fillna(df_split4['이유'])

df_split_result.count()

결론    952
그외    718
dtype: int64

In [23]:
df_split_result['결론'][1]

'\n원심판결 중 입학전형이의신청거부처분 취소청구 부분을 파기하되, 이 부분은 이 법원이 직접 재판하기에 충분하므로 자판하기로 하여 위 파기 부분에 해당하는 원고의 항소를 기각하고, 피고의 나머지 상고를 기각하며, 소송총비용 중 1/2은 원고가, 나머지는 피고가 각 부담하도록 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.\n상고이유를 판단한다.\n1. 사안의 개요\n원심판결 이유와 기록에 따르면 다음 사실을 알 수 있다.\n가. 원고는 제칠일안식일예수재림교(이하 ‘재림교’라 한다) 신자이다. 재림교는 금요일 일몰부터 토요일 일몰까지를 종교적 안식일로 여겨 직장·사업·학교 활동 및 시험 응시 등의 세속적 행위를 금지하고 있다.\n나. 피고는 2020. 4.경 ‘2021학년도 법학전문대학원 전문석사 입학전형 기본계획’(이하 ‘이 사건 기본계획’이라 한다)을, 2020. 6.경 ‘2021학년도 ○○대학교 법학전문대학원 신입생 모집요강’(이하 ‘이 사건 모집요강’이라 한다)을 각 공고하였다. 위 각 공고에 따르면 2021학년도 ○○대학교 법학전문대학원(이하 ‘○○대 법전원’이라 한다)의 입학생 선발은 서류전형으로 이루어진 1단계 평가를 거친 후 이에 합격한 학생들을 대상으로 면접평가와 논술평가를 실시하여 최종적으로 입학생을 선발하는 두 단계로 이루어진다. 면접평가는 토요일 오전반과 오후반으로 나누어 실시되고, 응시생들은 무작위로 각 면접반에 배정되는데, 면접 결시자는 불합격 처리하도록 되어 있다.\n다. 원고는 2021학년도 ○○대 법전원 전문석사 입학을 위한 입학원서를 제출하였는데, 면접일시가 토요일 일몰 전에 지정될 경우 안식일에 관한 원고의 종교적 신념을 지키면서 면접에 응시할 수 없었다. 이에 원고는 2020. 10.경 국가인권위원회에 대하여, 피고로 하여금 원고의 면접순서를 마지막으로 배치하는 등 원고의 종교적 양심을 제한하지 않을 수 있는 대체조치를 취할 것을 구하는 취지의 진정서를 제출하였다. 국가인권위원회는 2020. 10. 27. 피고에

#### 필요한 데이터 결합

In [24]:
df_concatenated = pd.concat([df_title1['title'], df_split5['판시사항'], df_split_result['결론']], axis=1)

In [25]:
print(df_concatenated.count())
df_concatenated

title    914
판시사항     960
결론       952
dtype: int64


,title,판시사항,결론
0,[보수약정금]〈북한주민인 피고들과 위임 및 보수약정을 체결한 법무법인인 원고가 위임...,[1] 북한주민이 상속 등으로 취득한 재산 그 자체는 물론이고 그 재산을 처분한 대...,"\n원심판결을 파기하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로 하여, ..."
1,[입학전형이의신청거부처분및불합격처분취소의소]〈종교적 신념을 이유로 면접시험 일정 변...,[1] 국립대학교 법학전문대학원 입시 과정에서 제칠일안식일예수재림교 신자들이 종교적...,"\n원심판결 중 입학전형이의신청거부처분 취소청구 부분을 파기하되, 이 부분은 이 법..."
2,[피해자보호명령등에대한재항고]〈피해자보호명령의 발령 요건 및 심리절차의 법령 위반이...,[1] 가정폭력범죄의 처벌 등에 관한 특례법에 도입된 피해자보호명령 제도의 취지 /...,"\n그러므로 원심결정을 취소하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로..."
3,[부당이득금]〈채무를 면하는 경우와 같은 재산의 소극적 증가도 부당이득이 성립하기 ...,[1] 채무를 면하는 경우와 같이 어떠한 사실의 발생으로 당연히 발생하였을 손실을 ...,\n상고이유를 판단한다.\n1. 법률상 원인 없이 타인의 재산 또는 노무로 인하여 ...
4,[성폭력범죄의처벌등에관한특례법위반(친족관계에의한강간)ㆍ아동복지법위반(아동학대)ㆍ아동...,[1] 형사소송법 제313조 제1항에서 규정한 ‘전 2조의 규정 이외에 피고인 또는...,"\n그러므로 상고를 모두 기각하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같..."
...,...,...,...
955,"[폭행]〈정당방위의 침해의 현재성 판단 기준이 문제된 사건〉 [공2023상,968]",정당방위의 요건 중 ‘침해의 현재성’의 의미 및 일련의 연속되는 행위로 인해 침해상...,"\n그러므로 원심판결 중 피고인 2에 대한 부분을 파기하고, 이 부분 사건을 다시 ..."
956,[임금]〈택시운수종사자가 유류비를 부담하는 약정은 무효라고 주장하면서 유류비 상당 ...,택시운송사업자의 운송비용 전가를 금지하는 구 택시운송사업의 발전에 관한 법률 제12...,"\n그러므로 상고를 기각하고 상고비용은 패소자가 부담하도록 하여, 대법관의 일치된 ..."
957,"[건물등철거]〈주한외국대사관 건물의 인접 토지 경계 침범 사건〉 [공2023상,894]",[1] 우리나라의 영토 내에서 행하여진 외국의 사법적(사법적) 행위에 대해 해당 국...,\n주위적 청구에 관한 원고의 상고를 일부 받아들이는 이상 예비적 청구에 관한 피고...
958,[근저당권말소]〈물상보증인이 피담보채무 잔액을 공탁하였음을 이유로 근저당권의 말소를...,[1] 채권자와 채무자 모두가 기한의 이익을 갖는 이자부 금전소비대차계약 등에 있어...,\n그러므로 나머지 상고이유에 대한 판단을 생략한 채 원심판결을 파기하고 사건을 다...


#### 결론 데이터를 못 추출해 낸 데이터 index를 구하여 inference할 데이터 추출

In [26]:
null_indices = df_concatenated[df_concatenated['결론'].isnull()].index.tolist()

In [27]:
raise ValueError()

ValueError: 

In [28]:
test = df_concatenated.iloc[null_indices]
test = test.drop(columns='결론')

In [29]:
test = test.dropna()

In [30]:
test.count()

title    8
판시사항     8
dtype: int64

In [31]:
test.to_json('./test_dataset.json', orient='records', lines = True, force_ascii=False)

#### None값을 제거하고 테스트 데이터 추출

In [32]:
df_concatenated = df_concatenated.dropna()


In [33]:
df_concatenated.count()

title    906
판시사항     906
결론       906
dtype: int64

In [34]:
df_concatenated.to_json('./train_dataset3.json', orient='records', lines = True, force_ascii=False)

In [35]:
df_concatenated

,title,판시사항,결론
0,[보수약정금]〈북한주민인 피고들과 위임 및 보수약정을 체결한 법무법인인 원고가 위임...,[1] 북한주민이 상속 등으로 취득한 재산 그 자체는 물론이고 그 재산을 처분한 대...,"\n원심판결을 파기하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로 하여, ..."
1,[입학전형이의신청거부처분및불합격처분취소의소]〈종교적 신념을 이유로 면접시험 일정 변...,[1] 국립대학교 법학전문대학원 입시 과정에서 제칠일안식일예수재림교 신자들이 종교적...,"\n원심판결 중 입학전형이의신청거부처분 취소청구 부분을 파기하되, 이 부분은 이 법..."
2,[피해자보호명령등에대한재항고]〈피해자보호명령의 발령 요건 및 심리절차의 법령 위반이...,[1] 가정폭력범죄의 처벌 등에 관한 특례법에 도입된 피해자보호명령 제도의 취지 /...,"\n그러므로 원심결정을 취소하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로..."
3,[부당이득금]〈채무를 면하는 경우와 같은 재산의 소극적 증가도 부당이득이 성립하기 ...,[1] 채무를 면하는 경우와 같이 어떠한 사실의 발생으로 당연히 발생하였을 손실을 ...,\n상고이유를 판단한다.\n1. 법률상 원인 없이 타인의 재산 또는 노무로 인하여 ...
4,[성폭력범죄의처벌등에관한특례법위반(친족관계에의한강간)ㆍ아동복지법위반(아동학대)ㆍ아동...,[1] 형사소송법 제313조 제1항에서 규정한 ‘전 2조의 규정 이외에 피고인 또는...,"\n그러므로 상고를 모두 기각하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같..."
...,...,...,...
955,"[폭행]〈정당방위의 침해의 현재성 판단 기준이 문제된 사건〉 [공2023상,968]",정당방위의 요건 중 ‘침해의 현재성’의 의미 및 일련의 연속되는 행위로 인해 침해상...,"\n그러므로 원심판결 중 피고인 2에 대한 부분을 파기하고, 이 부분 사건을 다시 ..."
956,[임금]〈택시운수종사자가 유류비를 부담하는 약정은 무효라고 주장하면서 유류비 상당 ...,택시운송사업자의 운송비용 전가를 금지하는 구 택시운송사업의 발전에 관한 법률 제12...,"\n그러므로 상고를 기각하고 상고비용은 패소자가 부담하도록 하여, 대법관의 일치된 ..."
957,"[건물등철거]〈주한외국대사관 건물의 인접 토지 경계 침범 사건〉 [공2023상,894]",[1] 우리나라의 영토 내에서 행하여진 외국의 사법적(사법적) 행위에 대해 해당 국...,\n주위적 청구에 관한 원고의 상고를 일부 받아들이는 이상 예비적 청구에 관한 피고...
958,[근저당권말소]〈물상보증인이 피담보채무 잔액을 공탁하였음을 이유로 근저당권의 말소를...,[1] 채권자와 채무자 모두가 기한의 이익을 갖는 이자부 금전소비대차계약 등에 있어...,\n그러므로 나머지 상고이유에 대한 판단을 생략한 채 원심판결을 파기하고 사건을 다...
